<a href="https://colab.research.google.com/github/JacobSchmitt/Basaa-Machine-Learning/blob/main/basaa/test/Basaa_MT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!wget -P '/content/baseline' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/convert.py
!wget -P '/content/baseline' https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/baseline/Makefile
!wget https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/evaluate.py
!wget https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/train.tsv 
!wget https://raw.githubusercontent.com/JacobSchmitt/Basaa-Machine-Learning/main/basaa/data/dev.tsv 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving corr_dev.tsv to corr_dev.tsv
Saving corr_train.tsv to corr_train.tsv


In [ ]:
%%capture
!pip install simpletransformers

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
train_df = pd.read_csv('corr_train.tsv', sep ='\t', names = ['input_text', 'target_text']).astype(str)
eval_df = pd.read_csv('corr_dev.tsv', sep ='\t', names = ['input_text', 'target_text']).astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

In [ ]:
def low(x):
  return x.lower()

In [ ]:
train_df.input_text = train_df.input_text.apply(low)
train_df.target_text = train_df.target_text.apply(low)

eval_df.input_text = eval_df.input_text.apply(low)
eval_df.target_text = eval_df.target_text.apply(low)

In [ ]:
train_df = train_df.dropna()
eval_df = eval_df.dropna()

In [ ]:
train_df = train_df.drop(train_df[train_df.input_text == '?'].index)
eval_df = eval_df.drop(eval_df[eval_df.input_text == '?'].index)

In [ ]:
import logging
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
# Check that we have a GPU
!nvidia-smi

Thu Apr  8 15:02:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
model_args = T5Args()
model_args.max_seq_length = 50
model_args.train_batch_size = 10
model_args.eval_batch_size = 10
model_args.num_train_epochs = 10
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = True
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = True
model_args.num_return_sequences = 1
# model_args.length_penalty = 3.0
# model_args.repetition_penalty = 2.0

model = T5Model("mt5", "google/mt5-small", args=model_args) 

INFO:filelock:Lock 140296555571792 acquired on /root/.cache/huggingface/transformers/97693496c1a0cae463bd18428187f9e9924d2dfbadaa46e4d468634a0fc95a41.dadce13f8f85f4825168354a04675d4b177749f8f11b167e87676777695d4fe4.lock


Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

INFO:filelock:Lock 140296555571792 released on /root/.cache/huggingface/transformers/97693496c1a0cae463bd18428187f9e9924d2dfbadaa46e4d468634a0fc95a41.dadce13f8f85f4825168354a04675d4b177749f8f11b167e87676777695d4fe4.lock
INFO:filelock:Lock 140294103203344 acquired on /root/.cache/huggingface/transformers/8e7b2a80ddcb5611b27d8c89e1e8e33a947e105415051402a22b9c8d7d1caeb0.e22331f3a065b885b30ae3dd1ff11ccaf7fbc444485f6eb07ef5e0138bca8b70.lock


Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

INFO:filelock:Lock 140294103203344 released on /root/.cache/huggingface/transformers/8e7b2a80ddcb5611b27d8c89e1e8e33a947e105415051402a22b9c8d7d1caeb0.e22331f3a065b885b30ae3dd1ff11ccaf7fbc444485f6eb07ef5e0138bca8b70.lock
INFO:filelock:Lock 140294103202960 acquired on /root/.cache/huggingface/transformers/37d0f67f084f8c5fc5589e0bba5ff3c6307af833bb0b7f4eb33fbfd8d4038a9d.84ea7af2df68dc8db434d3160aab65cce8ac63ce5b6f7743f8c9a4a14b4f77e2.lock


...

INFO:filelock:Lock 140294103202960 released on /root/.cache/huggingface/transformers/37d0f67f084f8c5fc5589e0bba5ff3c6307af833bb0b7f4eb33fbfd8d4038a9d.84ea7af2df68dc8db434d3160aab65cce8ac63ce5b6f7743f8c9a4a14b4f77e2.lock
INFO:filelock:Lock 140294103203216 acquired on /root/.cache/huggingface/transformers/685ac0ca8568ec593a48b61b0a3c272beee9bc194a3c7241d15dcadb5f875e53.f76030f3ec1b96a8199b2593390c610e76ca8028ef3d24680000619ffb646276.lock


Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

INFO:filelock:Lock 140294103203216 released on /root/.cache/huggingface/transformers/685ac0ca8568ec593a48b61b0a3c272beee9bc194a3c7241d15dcadb5f875e53.f76030f3ec1b96a8199b2593390c610e76ca8028ef3d24680000619ffb646276.lock
INFO:filelock:Lock 140294056555408 acquired on /root/.cache/huggingface/transformers/6a9e52d6dd21568e37b65fc180ada927968e8f7124f0acd6efcaf90cd2e0f4bb.4b81e5d952ad810ca1de2b3e362b9a26a5cc77b4b75daf20caf69fb838751c32.lock


Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

INFO:filelock:Lock 140294056555408 released on /root/.cache/huggingface/transformers/6a9e52d6dd21568e37b65fc180ada927968e8f7124f0acd6efcaf90cd2e0f4bb.4b81e5d952ad810ca1de2b3e362b9a26a5cc77b4b75daf20caf69fb838751c32.lock


In [ ]:
#  uncomment if you want to train the model yourself
model.train_model(train_df, eval_data=eval_df)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/9945 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
INFO:simpletransformers.t5.t5_model: Training started


Using Adafactor for T5


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/995 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
INFO:simpletransformers.t5.t5_model: Training of google/mt5-small model complete. Saved to outputs/.


(9950,
 {'eval_loss': [0.9710029470920563,
   0.629728479385376,
   0.5345365998148918,
   0.472048636674881,
   0.44343090042471883,
   0.4311791656911373,
   0.41250341847538946,
   0.40982581615448,
   0.4107170508801937,
   0.41716975122690203],
  'global_step': [995, 1990, 2985, 3980, 4975, 5970, 6965, 7960, 8955, 9950],
  'train_loss': [1.901863932609558,
   0.7343102097511292,
   0.6514549851417542,
   0.60931396484375,
   0.5508466362953186,
   0.48228156566619873,
   0.26619991660118103,
   0.4520193040370941,
   0.20957985520362854,
   0.27160099148750305]})

In [ ]:
# save the model
!zip -r "/content/model_73_4ep.zip" "/content/outputs/best_model" 

In [ ]:
# and move it to you Google Drive
!cp -r /content/model_73_4ep.zip /content/drive/MyDrive

In [ ]:
# load the model
from google.colab import drive
drive.mount('/content/drive')

!unzip "/content/drive/MyDrive/model_7epochs.zip" -d"/content/model"

Mounted at /content/drive
Archive:  /content/drive/MyDrive/model_7epochs.zip
   creating: /content/model/content/outputs/best_model/
  inflating: /content/model/content/outputs/best_model/pytorch_model.bin  
  inflating: /content/model/content/outputs/best_model/optimizer.pt  
  inflating: /content/model/content/outputs/best_model/config.json  
  inflating: /content/model/content/outputs/best_model/scheduler.pt  
  inflating: /content/model/content/outputs/best_model/model_args.json  
  inflating: /content/model/content/outputs/best_model/tokenizer_config.json  
  inflating: /content/model/content/outputs/best_model/training_args.bin  
 extracting: /content/model/content/outputs/best_model/eval_results.txt  
  inflating: /content/model/content/outputs/best_model/spiece.model  
  inflating: /content/model/content/outputs/best_model/special_tokens_map.json  


In [ ]:
import logging
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model_args = T5Args()
model_args.max_length = 512
model_args.length_penalty = 1
model_args.num_beams = 10
model_args.use_multiprocessed_decoding = True
model_args.use_multiprocessing = True
model_args.use_multiprocessing_for_evaluation = True

model = T5Model("mt5", "outputs/best_model", args=model_args) #change the second path to outputs if you trained the model and not loaded it "/content/model/content/outputs/best_model"

In [ ]:
sinhala_truth = eval_df['target_text'].tolist()

In [ ]:
sinhala_preds = model.predict(eval_df['input_text'].tolist())

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
df1 =  pd.DataFrame({'input': eval_df['input_text'], 'pred':sinhala_preds})

In [ ]:
eval_df.to_csv('test.tsv',sep='\t', index=False, header=False)

In [ ]:
def smart(x):
  double_e = 'e'+'́'
  double_o = 'o' + '̂'
  double_o_dots = 'o' + '̈'
  y = x.replace(double_e, 'é').replace(double_o, 'ô').replace(double_o_dots, 'ö')
  y = y.replace('▁', ' ')
  y = y.replace('_', ' ')
  y = y.capitalize()
  return y

In [ ]:
df1.pred = df1.pred.apply(smart)

In [ ]:
df1.to_csv('mt5_new.tsv',sep='\t', index=False, header=False)

In [ ]:
!python3 evaluate.py corr_dev.tsv mt5_new.tsv

CER: 23.637557452396585
WER: 51.99015941663275


In [ ]:
df1.head()

,input,pred
0,malét a mbôŋe hyémbi.,Màlêt à mɓòŋɛ hyembi.
1,a ntuhul me két (i) nuga.,À ǹtuhūl mɛ kēt (i) nugā.
2,"ñkum u ŋkôs ndoŋ, ŋkum u nlôôs ndoŋ.","Ŋ̀kùm u ŋkôs ndɔŋ, ŋ̀kùm u nlòòs ndɔŋ."
3,maŋ mut unu u nlo u mboda nyono wotama.,Màŋ mût unu u nlɔ̀ u mɓɔda nyɔ̀nɔ wɔtāma.
4,boŋge ba ŋke jogop ba kahal ndugi kôt maŋgudga.,Ɓɔ̀ŋgɛ ɓa ŋkɛ̀ jɔgɔ̂p ɓa kahal ndùgi kôt màŋgùdà.
